<a href="https://colab.research.google.com/github/IDCE-MSGIS/lab-4-zoning-maps-pandas-chikondi1/blob/main/Lab4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Add GIS packages to Google Colab
!pip install geopandas
!apt-get install -y libspatialindex-dev
!pip install rtree

#Import necessary packages and simplify names
import pandas as pd  #Provides interface for interacting with tabular data
import geopandas as gpd  #Combines the capabilities of pandas and shapely for geospatial operations
from shapely.geometry import Point, Polygon, MultiPolygon  #For manipulating text data into geospatial shapes
from shapely import wkt  #Stands for "well known text," which allows for interchange across GIS programs
import rtree  #Supports geospatial join

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive') #Connects Colab to Google Drive
root_path = 'gdrive/My Drive/gis/' #Sets root path to folder where data is stored

Mounted at /content/gdrive


In [ ]:
#Read in PUDs file as a geodataframe (Geopanda) and initialize coordinate reference system (CRS)
puds = gpd.read_file(root_path+'input/Planned_Unit_Development__PUDs.shp', crs = {'init' :'epsg:4326'})
aff = pd.read_csv(root_path+'input/Affordable_Housing.csv')
crosswalk = pd.read_csv(root_path+'input/zoning_crosswalk.csv')

#Displays rows from the first 3 index places (4 rows total)
puds.sample(3)

In [ ]:
#Provides a summary of what is within the Affordable housing .csv
aff.info()

In [ ]:
#Provides a summary of what is within the PUD .shp file
puds.info()

In [ ]:
#Creates a geometry column in the affordable housing dataframe.
aff['geometry'] = aff.apply(lambda row: Point(row.X, row.Y), axis=1)
aff = gpd.GeoDataFrame(aff, crs={'init' :'epsg:4326'}) #These numbers comprise a standard coordinate frame for the Earth

In [ ]:
aff.sample(1) #Sampling to the first index

In [ ]:
#Uses geospatial join to identify which PUDs include affordable housing projects.
puds_aff = gpd.sjoin(puds, aff, op='intersects', how='left')

In [ ]:
#Check our Merge - incremental development, we're checking as we go!
puds_aff.info()

In [ ]:
#Merges dataframe with zoning categories crosswalk to create the categories of "commercial", "residential" and "other/mixed use"
puds_info = puds_aff.merge(crosswalk[['Zone_Cat']], how='left', left_on='PUD_ZONING', right_on=crosswalk['Zone'])

In [ ]:
#Counts using print statements to show the total number of PUDs, and how many offer affordable housing.
print(f"Total count of PUDs: {puds_info.shape[0]}")
print(f"Count PUDs offering Affordable Housing: {puds_info.loc[~puds_info.PROJECT_NAME.isna()].shape[0]}")

In [ ]:
#Creates a map of PUDs by Zoning Category
puds_info.plot(column='Zone_Cat', legend=True, figsize=(16,8));

In [ ]:
#Creates a map of the PUDs that provide Affordable Housing
puds_info[puds_info['TOTAL_AFFORDABLE_UNITS']>0].plot(column='TOTAL_AFFORDABLE_UNITS', color='grey', figsize=(16,8));

In [ ]:
#Exports geodataframe as shapefile within Colab
puds_info.to_file('puds_info.shp')

#Copies shapefile components from Colab to Google Drive using bash script
!cp puds_info.cpg 'gdrive/My Drive/gis/output' #Copies to the output folder
!cp puds_info.dbf 'gdrive/My Drive/gis/output'
!cp puds_info.prj 'gdrive/My Drive/gis/output'
!cp puds_info.shp 'gdrive/My Drive/gis/output'
!cp puds_info.shx 'gdrive/My Drive/gis/output'